# Building and Validating an Experiment Configuration

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.5.0.</div>

This notebook demonstrates how to build up a configuration file for generating FV3 initial conditions (ICs) from a hierarchy of smaller, purpose-specific files; dereferencing Jinja2 expressions in the configuration; and validating the final configuration to check for potential errors. A larger, more complex experimental setup could be built up by applying similar techniques.
<!--cell 0-->

In [1]:
from datetime import datetime, timedelta
from uwtools.api.config import get_yaml_config
from uwtools.api import chgres_cube
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

We start with a base file that configures the `chgres_cube` component to generate FV3 ICs for use with the default physics suite, controlled by the `varmap_file` key:
<!--cell 2-->

In [2]:
%%bash
cat fixtures/exp-config/base-file.yaml

task_make_ics:
  chgres_cube:
    execution:
      executable: "execdir/chgres_cube"
    namelist:
      update_values:
        config:
          cycle_day: !int "{{ cycle.strftime('%d') }}"
          cycle_hour: !int "{{ cycle.strftime('%H') }}"
          cycle_mon: !int "{{ cycle.strftime('%m') }}"
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: "{{ task_make_ics.chgres_cube.rundir }}"
          external_model: "GFS"
          input_type: "gaussian_nemsio"
          mosaic_file_target_grid: "path/to/example_mosaic.halo.nc"
          tg3_from_soil: false
          tracers:
            - sphum
            - liq_wat
          tracers_input:
            - spfh
            - clwmr
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GFSphys_var_map.txt"
          vcoord_file_target_grid: "path/to/global_hyblev.165.txt"
    rundir: '{{ workflow.EXPTDIR }}/make_ics'


To produce ICs compatible with the FV3_RAP physics suite instead, this partial configuration can be used to update the base:
<!--cell 4-->

In [3]:
%%bash
cat fixtures/exp-config/fv3-rap-physics.yaml

task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GSDphys_var_map.txt"


User- and experiment-specific values can be supplied via a third configuration file:
<!--cell 6-->

In [4]:
%%bash
cat fixtures/exp-config/user.yaml

user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-srweather-app/parm
workflow: 
  EXPTDIR: /path/to/my/output


Structuring the configuration as a hierarchy of increasing specificity provides a better user experience through separation of concerns: Users can see why certain values are changing, and can mix together app-supplied fragments with known-good values into larger experiment configurations.

Here, we start by instantiating a `YAMLConfig` object from the most general base config file, which contains unrendered Jinja2 expressions and is missing certain user- and experiment-specific values:
<!--cell 8-->

In [5]:
experiment_config = get_yaml_config('fixtures/exp-config/base-file.yaml')
print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: '{{ task_make_ics.chgres_cube.rundir }}'
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: '{{ user.PARMdir }}/ufs_utils/varmap_tables/GFSphys_var_map.txt'
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: '{{ workflow.EXPTDIR }}/make_ics'


Next, we define a list of additional config files, iterate over those, and update the base config with each, in turn. Note that, if the configs share any keys, the values from the update will override and replace existing ones. For example, the original `varmap_file:` path to file `GFSphys_var_map.txt` is updated with a path to file `GSDphys_var_map.txt`:
<!--cell 10-->

In [6]:
config_files = [
    'fixtures/exp-config/fv3-rap-physics.yaml',
    'fixtures/exp-config/user.yaml'
]
for config_file in config_files:
    config = get_yaml_config(config_file)
    experiment_config.update_from(config)

print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: '{{ task_make_ics.chgres_cube.rundir }}'
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: '{{ user.PARMdir }}/ufs_utils/varmap_tables/GSDphys_var_map.txt'
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: '{{ workflow.EXPTDIR }}/make_ics'
user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/t

Once the hierarchy of configs is merged, we call the `dereference()` method to render Jinja2 expressions into final values. Keys like `varmap_file:` and `rundir:` have their values rendered using references to the `PARMdir` and `EXPTDIR` keys in the `user` and `workflow` sections, respectively. Expressions with cycle-specific references remain, and will be rendered at run time.
<!--cell 12-->

In [7]:
experiment_config.dereference()
print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: /path/to/my/output/make_ics
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: /path/to/ufs-srweather-app/parm/ufs_utils/varmap_tables/GSDphys_var_map.txt
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: /path/to/my/output/make_ics
user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-sr

To catch potential configuration errors as early as possible, the `uwtools` driver for `chgres_cube` is called to validate the config using a built-in schema. The driver requires a `cycle` parameter with a `datetime` value, and the current time is used here. As the output shows, no schema-validation errors are found
<!--cell 14-->

In [8]:
driver = chgres_cube.ChgresCube(
    config=experiment_config,
    key_path=['task_make_ics'],
    cycle=datetime.now(),
    leadtime=timedelta(hours=6),
)
driver.validate()

[2024-11-19T23:14:15]     INFO Validating config against internal schema: chgres-cube
[2024-11-19T23:14:15]     INFO 0 schema-validation errors found in chgres_cube config
[2024-11-19T23:14:15]     INFO Validating config against internal schema: platform
[2024-11-19T23:14:15]     INFO 0 schema-validation errors found in platform config
[2024-11-19T23:14:15]     INFO 20241120 05:14:15 chgres_cube valid schema: State: Ready


Asset(ref=None, ready=<function Assets.validate.<locals>.<lambda> at 0xffff685ad6c0>)